In [10]:
import numpy as np

# Load keypoints and labels
X = np.load("sign_keypoints.npy")  # Features (hand keypoints)
y = np.load("sign_labels.npy")  # Labels (sign classes)

print(f"Loaded {X.shape[0]} samples with {X.shape[1]} features each.")
print(f"Unique classes: {np.unique(y)}")

Loaded 105420 samples with 63 features each.
Unique classes: ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h'
 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [11]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Reshape X to fit LSTM (sequence length = 1 since each frame is independent)
X = X.reshape(X.shape[0], 1, X.shape[1])  # Shape: (samples, sequence_length, features)

# Encode labels to numbers (A → 0, B → 1, ..., Z → 25)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# One-hot encode labels for training
y_categorical = to_categorical(y_encoded)

print(f"X shape: {X.shape}, y shape: {y_categorical.shape}")


X shape: (105420, 1, 63), y shape: (105420, 36)


In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define the LSTM model
model = Sequential([
    LSTM(64, return_sequences=False, input_shape=(1, X.shape[2])),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(y_categorical.shape[1], activation='softmax')  # Output layer with softmax
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                   │ (None, 64)             │        32,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 36)             │         1,188 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,036 (140.77 KB)

 Trainable params: 36,036 (140.77 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Train the model
history = model.fit(X, y_categorical, epochs=20, batch_size=16, validation_split=0.2)

# Save the trained model
model.save("asl_sign_language_model.h5")

print("✅ Model trained and saved as 'asl_sign_language_model.h5'")


Epoch 1/20
5271/5271 ━━━━━━━━━━━━━━━━━━━━ 5s 829us/step - accuracy: 0.0637 - loss: 3.2806 - val_accuracy: 0.0000e+00 - val_loss: 6.4605
Epoch 2/20
5271/5271 ━━━━━━━━━━━━━━━━━━━━ 4s 822us/step - accuracy: 0.0647 - loss: 3.0568 - val_accuracy: 0.0000e+00 - val_loss: 8.6044
Epoch 3/20
5271/5271 ━━━━━━━━━━━━━━━━━━━━ 5s 852us/step - accuracy: 0.0645 - loss: 3.0494 - val_accuracy: 0.0000e+00 - val_loss: 10.7153
Epoch 4/20
5271/5271 ━━━━━━━━━━━━━━━━━━━━ 5s 863us/step - accuracy: 0.0672 - loss: 3.0491 - val_accuracy: 0.0000e+00 - val_loss: 12.8164
Epoch 5/20
5271/5271 ━━━━━━━━━━━━━━━━━━━━ 5s 872us/step - accuracy: 0.0666 - loss: 3.0492 - val_accuracy: 0.0000e+00 - val_loss: 14.8406
Epoch 6/20
5271/5271 ━━━━━━━━━━━━━━━━━━━━ 5s 890us/step - accuracy: 0.0643 - loss: 3.0510 - val_accuracy: 0.0000e+00 - val_loss: 16.4714
Epoch 7/20
5271/5271 ━━━━━━━━━━━━━━━━━━━━ 5s 885us/step - accuracy: 0.0663 - loss: 3.0501 - val_accuracy: 0.0000e+00 - val_loss: 17.4263
Epoch 8/20
5271/5271 ━━━━━━━━━━━━━━━━━━━━ 5

✅ Model trained and saved as 'asl_sign_language_model.h5'


In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional, Attention, LayerNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Define learning rate schedule
lr_schedule = ExponentialDecay(initial_learning_rate=0.001, decay_steps=1000, decay_rate=0.9, staircase=True)

# Define the upgraded LSTM model
model = Sequential([
    # First BiLSTM Layer
    Bidirectional(LSTM(256, return_sequences=True, kernel_regularizer=l2(0.001))),
    BatchNormalization(),
    Dropout(0.3),

    # Second BiLSTM Layer
    Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.001))),
    BatchNormalization(),
    Dropout(0.3),

    # Attention Mechanism
    Attention(),

    # Third LSTM Layer (Single Direction)
    LSTM(64, return_sequences=False, kernel_regularizer=l2(0.001)),
    LayerNormalization(),
    Dropout(0.3),

    # Dense Layers with Swish Activation
    Dense(128, activation='swish', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(64, activation='swish', kernel_regularizer=l2(0.001)),
    Dropout(0.3),

    # Output Layer
    Dense(y_categorical.shape[1], activation='softmax')
])

# Compile the model with a learning rate schedule
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model summary
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_1 (Attention)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_1           │ ?                      │   0 (unbuilt) │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [16]:
import tensorflow as tf
import numpy as np

# Load existing model
model = tf.keras.models.load_model("asl_sign_language_model_v2.h5")

# Load dataset
X_train = np.load("sign_keypoints.npy")
y_train = np.load("sign_labels_encoded.npy")

# Reshape input for LSTM
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))  # (samples, sequence_length, features)

# Compile the model again (optional)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train for additional epochs
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)


# Save the fine-tuned model
model.save("asl_sign_language_model_v3.h5")

print("✅ Model fine-tuned and saved as 'asl_sign_language_model_v2.h5'")


Epoch 1/50
1318/1318 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.0644 - loss: 3.0490 - val_accuracy: 0.0000e+00 - val_loss: 21.8305
Epoch 2/50
1318/1318 ━━━━━━━━━━━━━━━━━━━━ 1s 966us/step - accuracy: 0.0668 - loss: 3.0488 - val_accuracy: 0.0000e+00 - val_loss: 21.8326
Epoch 3/50
1318/1318 ━━━━━━━━━━━━━━━━━━━━ 1s 969us/step - accuracy: 0.0652 - loss: 3.0495 - val_accuracy: 0.0000e+00 - val_loss: 21.8344
Epoch 4/50
1318/1318 ━━━━━━━━━━━━━━━━━━━━ 1s 983us/step - accuracy: 0.0636 - loss: 3.0479 - val_accuracy: 0.0000e+00 - val_loss: 21.8358
Epoch 5/50
1318/1318 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0663 - loss: 3.0489 - val_accuracy: 0.0000e+00 - val_loss: 21.8388
Epoch 6/50
1318/1318 ━━━━━━━━━━━━━━━━━━━━ 1s 988us/step - accuracy: 0.0653 - loss: 3.0495 - val_accuracy: 0.0000e+00 - val_loss: 21.8410
Epoch 7/50
1318/1318 ━━━━━━━━━━━━━━━━━━━━ 1s 998us/step - accuracy: 0.0641 - loss: 3.0510 - val_accuracy: 0.0000e+00 - val_loss: 21.8433
Epoch 8/50
1318/1318 ━━━━━━━━━━━━━━━━━━━━ 1s 

✅ Model fine-tuned and saved as 'asl_sign_language_model_v2.h5'


In [8]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy Over Time')
plt.show()

# Plot training & validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss Over Time')
plt.show()


NameError: name 'history' is not defined

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, LSTM, Dense, Dropout, Bidirectional, LayerNormalization,
    MultiHeadAttention, Add, SpatialDropout1D
)
from tensorflow.keras.optimizers.schedules import CyclicalLearningRate

# Cyclic Learning Rate Schedule (Smarter LR Adjustment)
clr = CyclicalLearningRate(
    initial_learning_rate=1e-5,
    maximal_learning_rate=5e-4,
    step_size=2000,
    scale_fn=lambda x: 0.5 * (1 + tf.math.cos(x * 3.1416))
)

# Model Input Shape
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))

# 🔥 BiLSTM Base Layer (Context Extraction)
x = Bidirectional(LSTM(256, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.0001)))(input_layer)
x = LayerNormalization()(x)
x = SpatialDropout1D(0.3)(x)

# 🔥 Multi-Head Self-Attention (Focuses on Important Features)
attention = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
x = Add()([x, attention])  # Residual Connection
x = LayerNormalization()(x)

# 🔥 Second BiLSTM Layer (Deeper Temporal Understanding)
x = Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.0001)))(x)
x = LayerNormalization()(x)
x = SpatialDropout1D(0.3)(x)

# 🔥 Final LSTM Layer (Feature Compression)
x = LSTM(64, return_sequences=False, kernel_regularizer=tf.keras.regularizers.l2(0.0001))(x)
x = LayerNormalization()(x)

# 🔥 Dense Layers with Swish Activation
x = Dense(128, activation='swish')(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='swish')(x)
x = Dropout(0.3)(x)

# Output Layer (Softmax for Classification)
output_layer = Dense(y_train.shape[1], activation='softmax')(x)

# Define & Compile Model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=clr, clipnorm=1.0),  # Adaptive Gradient Clipping
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Model Summary
model.summary()


ImportError: cannot import name 'CyclicalLearningRate' from 'tensorflow.keras.optimizers.schedules' (C:\Users\PC\PycharmProjects\tstai\.venv\Lib\site-packages\keras\_tf_keras\keras\optimizers\schedules\__init__.py)